In [ ]:
pip install azure-storage-blob


In [ ]:


import requests
from bs4 import BeautifulSoup
import pandas as pd


url = "https://www.flipkart.com/search?q=mobile+phones&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_1_6_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_6_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=mobile+phones&requestId=76320354-c6bd-4d5e-aff8-638d6a1daa8d&as-backfill=on"  # Replace as needed


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}


response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")


titles, prices, ratings, reviews = [], [], [], []


products = soup.find_all("div", class_="tUxRFH")  
for product in products:

    title_tag = product.find("div", class_="KzDlHZ")
    title = title_tag.get_text(strip=True) if title_tag else "N/A"
    titles.append(title)

 
    price_tag = product.find("div", class_="Nx9bqj")
    price = price_tag.get_text(strip=True).replace("₹", "").replace(",", "") if price_tag else "N/A"
    prices.append(price)


    rr_tag = product.find("span", class_="Wphh3N")
    if rr_tag:
        rr_text = rr_tag.get_text(strip=True)
        try:
            rating_part = rr_text.split("Ratings")[0].replace(",", "").strip()
            review_part = rr_text.split("Reviews")[0].split("&")[-1].strip().replace(",", "")
        except:
            rating_part, review_part = "N/A", "N/A"
    else:
        rating_part, review_part = "N/A", "N/A"

    ratings.append(rating_part)
    reviews.append(review_part)


df = pd.DataFrame({
    "Title": titles,
    "Price (INR)": prices,
    "Ratings": ratings,
    "Reviews": reviews
})


df.to_csv("flipkart_products.csv", index=False)

print(f"Scraped {len(df)} products and saved to 'flipkart_products.csv'")
df.head()
